<a href="https://colab.research.google.com/github/eleven111101/GenAI_JPN11/blob/main/pages_extract.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install opencv-python-headless
!pip install pytesseract
!pip install pdf2image
!pip install pandas openpyxl
!sudo apt-get install tesseract-ocr

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [ ]:
!sudo apt-get install poppler-utils


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
poppler-utils is already the newest version (22.02.0-2ubuntu0.4).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [ ]:
import pandas as pd
import cv2
import pytesseract
from pdf2image import convert_from_path
import os

# Add Poppler binaries to the PATH
os.environ["PATH"] += os.pathsep + "/usr/bin"

def extract_text_and_images(pdf_path, start_page, end_page):
    extracted_data = []

    # Convert PDF pages to images using pdf2image
    pages = convert_from_path(pdf_path, 300, first_page=start_page, last_page=end_page)  # Set the DPI and page range

    for i, page in enumerate(pages, start=start_page):
        # Save the image temporarily
        temp_image_path = f"temp_page_{i}.png"
        page.save(temp_image_path, "PNG")

        # Read the saved image using OpenCV
        img = cv2.imread(temp_image_path)

        if img is not None:
            # Use pytesseract to perform OCR on the preprocessed image
            img_text = pytesseract.image_to_string(img, config='--psm 6')  # PSM 6 for sparse text

            # Split the extracted text into separate lines
            extracted_lines = img_text.strip().split('\n')

            # Append extracted lines and image to the list
            extracted_data.append({
                'Page': i,
                'Text': '\n'.join(extracted_lines),
                'Image': cv2.imencode('.png', img)[1]  # Encode image as PNG bytes
            })

        # Remove the temporary image file
        os.remove(temp_image_path)

    return extracted_data

try:
    # Ask the user for the input PDF file
    pdf_path = input("Enter the path to the PDF file: ")

    # Check if the file exists
    if not os.path.isfile(pdf_path):
        print("Error: The specified file does not exist.")
    else:
        # Ask for the start and end page numbers
        start_page = int(input("Enter the start page number: "))
        end_page = int(input("Enter the end page number: "))

        # Extract text and images from the specified page range
        extracted_data = extract_text_and_images(pdf_path, start_page, end_page)

        # Create a Pandas DataFrame from the extracted data
        df = pd.DataFrame(extracted_data)

        # Create the output file name
        output_file = f"{start_page}-{end_page}_extracted_data.xlsx"

        # Create a Pandas ExcelWriter object
        with pd.ExcelWriter(output_file) as writer:
            # Write data to Excel
            df.to_excel(writer, sheet_name='Extracted_Data', index=False)

        print(f"Data extracted and saved to {output_file}.")

except Exception as e:
    print(f"An error occurred: {e}")


Enter the path to the PDF file: /content/euro-ncap-assessment-protocol-sa-v903.pdf
Enter the start page number: 4
Enter the end page number: 6
